In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from functools import partial

# Load Data

In [11]:
data = pd.read_csv('../data/model_comp_rt_pred.csv', sep=';', header=0)

# MMD Functions

In [12]:
def maximum_mean_discrepancy(source_samples, target_samples, minimum=0., unbiased=False):
    """ This Maximum Mean Discrepancy (MMD) loss is calculated with a number of different Gaussian kernels.

    """

    sigmas = [
        1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15, 20, 25, 30, 35, 100,
        1e3, 1e4, 1e5, 1e6
    ]
    gaussian_kernel = partial(_gaussian_kernel_matrix, sigmas=sigmas)
    if unbiased:
        loss_value = _mmd_kernel_unbiased(source_samples, target_samples, kernel=gaussian_kernel)
    else:
        loss_value = _mmd_kernel(source_samples, target_samples, kernel=gaussian_kernel)
        
        
    loss_value = tf.maximum(minimum, loss_value) 
    return loss_value

def _gaussian_kernel_matrix(x, y, sigmas):
    """ Computes a Gaussian Radial Basis Kernel between the samples of x and y.

    We create a sum of multiple gaussian kernels each having a width :math:`\sigma_i`.

    Parameters
    ----------
    x :  tf.Tensor of shape (M, num_features)
    y :  tf.Tensor of shape (N, num_features)
    sigmas : list(float)
        List which denotes the widths of each of the gaussians in the kernel.

    Returns
    -------
    kernel: tf.Tensor
        RBF kernel of shape [num_samples{x}, num_samples{y}]
    """
    def norm(v):
        return tf.reduce_sum(tf.square(v), 1)
    beta = 1. / (2. * (tf.expand_dims(sigmas, 1)))
    dist = tf.transpose(norm(tf.expand_dims(x, 2) - tf.transpose(y)))
    s = tf.matmul(beta, tf.reshape(dist, (1, -1)))
    kernel = tf.reshape(tf.reduce_sum(tf.exp(-s), 0), tf.shape(dist))
    return kernel

def _mmd_kernel(x, y, kernel=_gaussian_kernel_matrix):
    """ Computes the Maximum Mean Discrepancy (MMD) of two samples: x and y.

    Maximum Mean Discrepancy (MMD) is a distance-measure between the samples of the distributions of x and y.

    Parameters
    ----------
    x      : tf.Tensor of shape (num_samples, num_features)
    y      : tf.Tensor of shape (num_samples, num_features)
    kernel : callable, default: _gaussian_kernel_matrix
        A function which computes the kernel in MMD.

    Returns
    -------
    loss : tf.Tensor
        squared maximum mean discrepancy loss, shape (,)
    """

    loss = tf.reduce_mean(kernel(x, x))  # lint error: sigmas unfilled
    loss += tf.reduce_mean(kernel(y, y))  # lint error: sigmas unfilled
    loss -= 2 * tf.reduce_mean(kernel(x, y))  # lint error: sigmas unfilled
    return loss

def _mmd_kernel_unbiased(x, y, kernel=_gaussian_kernel_matrix):
    """ Computes the Maximum Mean Discrepancy (MMD) of two samples: x and y.

    Maximum Mean Discrepancy (MMD) is a distance-measure between the samples of the distributions of x and y.

    Parameters
    ----------
    x      : tf.Tensor of shape (num_samples, num_features)
    y      : tf.Tensor of shape (num_samples, num_features)
    kernel : callable, default: _gaussian_kernel_matrix
        A function which computes the kernel in MMD.

    Returns
    -------
    loss : tf.Tensor
        squared maximum mean discrepancy loss, shape (,)
    """
    m, n = x.shape[0], y.shape[0]
    loss = (1.0/(m*(m+1))) * tf.reduce_sum(kernel(x, x))  # lint error: sigmas unfilled
    loss += (1.0/(n*(n+1))) * tf.reduce_sum(kernel(y, y))  # lint error: sigmas unfilled
    loss -= (2.0/(m*n)) * tf.reduce_sum(kernel(x, y))  # lint error: sigmas unfilled
    return loss

In [13]:
def mmd_permutation(x, y, n_perm=1000, kernel=_gaussian_kernel_matrix, unbiased=False):
    """
    Computes the p-value of the MMD by permuting the samples.
    """
    
    # Obtain sample sizes
    m = int(x.shape[0])
    n = int(y.shape[0])
    xy = tf.concat((x, y), axis=0)
    
    # Prepare MMD
    sigmas = [
        1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15, 20, 25, 30, 35, 100,
        1e3, 1e4, 1e5, 1e6
    ]
    gaussian_kernel = partial(_gaussian_kernel_matrix, sigmas=sigmas)
    mmd2_null = np.zeros(n_perm)
    
    # Run permutations
    for i in range(n_perm):
        idx = np.random.permutation(m+n)
        xy = tf.gather(xy, idx, axis=0)
        if unbiased:
            mmd2 = _mmd_kernel_unbiased(xy[:m], xy[m:], kernel=gaussian_kernel)
        else:
            mmd2 = _mmd_kernel(xy[:m], xy[m:], kernel=gaussian_kernel)
        mmd2_null[i] = mmd2
    return mmd2_null

# Compute MMDs

## Extract Data

In [34]:
x_dm = np.atleast_2d(data['rt_fast_dm'].values).T.astype(np.float32)
x_bf = np.atleast_2d(data['rt_dynamic'].values).T.astype(np.float32)
x_em = np.atleast_2d(data['rt_empiric'].values).T.astype(np.float32)

## Compute Critical MMD and Permutation Test

In [ ]:
mmd_dm_em = maximum_mean_discrepancy(x_dm, x_em)
mmd_dm_em_p = mmd_permutation(x_dm, x_em)

In [41]:
mmd_bf_em = maximum_mean_discrepancy(x_bf, x_em)
mmd_bf_em_p = mmd_permutation(x_bf, x_em)

<tf.Tensor: shape=(), dtype=float32, numpy=0.013235092>

In [42]:
mmd_bf_dm = maximum_mean_discrepancy(x_bf, x_dm)
mmd_bf_dm_p = mmd_permutation(x_bf, x_dm)

<tf.Tensor: shape=(), dtype=float32, numpy=0.015806198>